In [37]:
import pandas as pd
from ortools.linear_solver import pywraplp
import numpy as np

dataset = pd.read_csv("../Downloads/truc/faces.utf8.csv")
dataset = dataset.values
briefs = pd.read_csv("../Downloads/truc/briefs.utf8.csv")
briefs = briefs.values

def get_week(week, data):
    array = []
    for i in range(0, len(data)):
        if data[i][0] == week:
            array.append(data[i])
    return (array)

dict_col = {}
dict_col["25_49csp"] = 12
dict_col["cspp"] = 15
dict_col["urb"] = 18
dict_col["all"] = 14
dict_col["f25_49rda"] = 17
dict_col["f25_49csp"] = 16
dict_col["15_24"] = 10
dict_col["25_49"] = 11
dict_col["25_49urb"] = 13

def get_max_week():
    return (briefs[len(briefs)-1][0])

def solve_week(week_nb):
    brief = [[]]
    full_week = get_week(week_nb, briefs)
    data = dataset.tolist()
    for k in range (0, len(full_week)):
        brief[0] = full_week[k]
        solver = pywraplp.Solver('SolveStigler', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

        faces = [[]] * len(data)
        objective = solver.Objective()
        for i in range(0, len(data)):
            faces[i] = solver.NumVar(0, 1, data[i][0])
            objective.SetCoefficient(faces[i], 1)
        objective.SetMinimization()

        constraints = [0] * len(brief)
        for i in range(0, len(brief)):
            constraints[i] = solver.Constraint(brief[i][3], solver.infinity())
            for j in range(0, len(data)):
                contactkey = dict_col[brief[i][2]]
                constraints[i].SetCoefficient(faces[j], data[j][contactkey])
                

        status = solver.Solve()
        if status == solver.OPTIMAL:
            result = []
            nb_faces = 0
            print('Optimal Solution Found')
            for i in range (0, len(data)):
                ref = []
                if (faces[i].solution_value() > 0):
                    nb_faces += 1
                    ref = data.pop(i)
                    result.append(ref[0])
            print (len(data))
            print("week " + str(week_nb) + ", brief n°" + str(k) + " : using " + str(nb_faces) + " faces")
        else:
            if status == solver.FEASIBLE:
                print('Solution Found, but not Optimal')
            if status != solver.FEASIBLE:
                print('The solver could not solve the problem')

for i in range(1, get_max_week() + 1):
    solve_week(i)

Optimal Solution Found
121048
week 1, brief n°0 : using 256 faces
Optimal Solution Found
120788
week 1, brief n°1 : using 260 faces
Optimal Solution Found
120208
week 1, brief n°2 : using 580 faces
The solver could not solve the problem
The solver could not solve the problem
Optimal Solution Found
120178
week 1, brief n°5 : using 30 faces
The solver could not solve the problem
Optimal Solution Found
119522
week 1, brief n°7 : using 656 faces
The solver could not solve the problem
The solver could not solve the problem
Optimal Solution Found
118668
week 1, brief n°10 : using 854 faces
The solver could not solve the problem
Optimal Solution Found
118310
week 1, brief n°12 : using 358 faces
Optimal Solution Found
118068
week 1, brief n°13 : using 242 faces
The solver could not solve the problem
The solver could not solve the problem
Optimal Solution Found
118029
week 1, brief n°16 : using 39 faces
Optimal Solution Found
117397
week 1, brief n°17 : using 632 faces
Optimal Solution Found
11

IndexError: pop index out of range